<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/12-Geospatial-Data/12.3%20Choropleths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 12.3 Choropleths

A **choropleth** is a map in which areas are colored according to some statistic of interest. Perhaps the most familiar example of a choropleth is the presidential election map, which shows the percentage in each county who voted for the Democratic or Republican candidate. In this graphic, the areal units are counties, and the statistic of interest is the percentage who voted for the Democratic (or Republican) candidate. 

![](https://github.com/dlsun/pods/blob/master/12-Geospatial-Data/img/2016election.png?raw=1)

In this lesson, you will learn how to make choropleths like the one above.

## Shapefiles

The shapefile format is a data format for geometric objects, such as points, lines, and polygons. A shapefile can be used to describe the boundaries of a lake, the course of a river, or the boundaries of a county.

You can find shapefiles for most geographic entities online. For example, the [U.S. Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html) maintains shapefiles for boundaries of states, counties, and congressional districts in the United States. Shapefiles for international data can be found [at the Natural Earth website](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/).

The code below downloads the U.S. county shapefiles at the lowest possible resolution (1:20,000,000).

In [0]:
!wget https://www2.census.gov/geo/tiger/GENZ2016/shp/cb_2016_us_county_20m.zip
!unzip cb_2016_us_county_20m.zip -d counties
!rm cb_2016_us_county_20m.zip
!ls counties/

Notice that "shapefile" is somewhat of a misnomer, as the format refers not to a single file but a collection of files, all of which have the same filename but different extensions. The main extensions are:

- `.shp` - shape format, which stores the geometric objects
- `.shx` - shape index format, which indexes the objects to make them quickly searchable
- `.dbf` - attribute format, which stores additional metadata about each object
- `.prj` - projection format

To read in a shapefile using Basemap, we first set up the map, then call the `.readshapefile()` method, which takes two arguments: 

1. the common filename of the shapefiles, not including the file extension
2. a name for the field that will store the attributes (you can pick any name you like, but try to be descriptive).

In [0]:
!apt-get -qq install libgeos-dev
!pip install -qq https://github.com/matplotlib/basemap/archive/master.zip

In [0]:
from mpl_toolkits.basemap import Basemap

m = Basemap(llcrnrlon=-119, llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49,
            projection="lcc", lat_1=33, lat_2=45, lon_0=-95)
m.readshapefile("counties/cb_2016_us_county_20m", "counties")

Since the field was named `counties`, the vertices of each polygon will be stored in `m.counties`

In [0]:
m.counties[2]

and information about each county stored in `m.counties_info`.

In [0]:
m.counties_info[:2]

Note that the information in `m.counties_info` can be readily converted into a `DataFrame`.

In [0]:
import pandas as pd

df_counties = pd.DataFrame(m.counties_info)
df_counties

## Patches and Collections in `matplotlib`

In order to understand how choropleths are made, it is important to first understand what a `Patch` is in `matplotlib`. A `Patch` is simply a 2-dimensional object with an edge color and face color. Examples of `Patch`es include `Circle`s, `Rectangle`s, and `Polygon`s. Since areal units---like countries, states, and counties---are polygons in general, the most important type of `Patch` for making a choropleth is the `Polygon`. A `Polygon` is specified by a list of its vertices. For example, the following code creates a red `Polygon` out of the first county in the shapefile:

In [0]:
import matplotlib as mpl

polygon = mpl.patches.Polygon(m.counties[0], facecolor="red")
polygon

To draw the `Polygon` object that we just constructed, we first initialize a map and add the `Polygon` patch to the map. Notice that it is colored red, just like we specified above.

In [0]:
import matplotlib.pyplot as plt

# Initialize a blank map.
m.drawmapboundary()
# Get the current axes.
ax = plt.gca()
# Add the patch to the current axes.
ax.add_patch(polygon)

Thus, one way to create a choropleth is to draw the `Polygon` for each county, one at a time, setting the face color of each patch to an appropriate color based on the data value for that county. This requires that we maintain a color map that maps data values to colors.

An easier way to make a choropleth is to create a `PatchCollection`, which allows us to associate a value with each patch and automatically generates a color map from the values. For example, suppose we want to color each county according to the fraction of the county that is water.

In [0]:
frac_water = df_counties["AWATER"] / (df_counties["AWATER"] + df_counties["ALAND"])
frac_water

We first create a `Polygon` for each county and then create a `PatchCollection` out of the polygons. Since we are displaying the fraction of each county that is _water_, it makes sense to use a blue color map (`Blues`). (A list of the available colormaps can be found [here](https://matplotlib.org/users/colormaps.html).)

In [0]:
# Turn each set of vertices into a matplotlib Polygon.
polygons = [mpl.patches.Polygon(vertices) for vertices in m.counties]

# Create a collection of Polygons.
collection = mpl.collections.PatchCollection(polygons, cmap="Blues")

Now for the key step. We have to tell the `PatchCollection` about the data values. To do this, we use the `.set_array()` method. Notice that the data values are in the same order as the polygons; the former was derived from `m.counties_info`, while the latter was derived from `m.counties`. The shapefile guarantees that these two lists are aligned.

In [0]:
collection.set_array(frac_water)

Now we are ready to draw the map. To add a `PatchCollection` to a `matplotlib` `AxesSubplot` object, we use the `.add_collection()` method.

In [0]:
# Initialize a blank map.
m.drawmapboundary()
# Get the current axes.
ax = plt.gca()
# Add the collection to the current axes.
ax.add_collection(collection)

Notice that the counties that border major bodies of water tend to have a higher fraction of their area as water. This makes intuitive sense.

## Making Choropleths with Outside Data

In the above example, we made a choropleth from data that was already in the shapefile. But in general, the shapefile only contains minimal metadata about each areal unit. Suppose the data that we want to visualize resides in a separate file. For example, suppose we want to plot the 2016 presidential election results by county.

In [0]:
data_dir = "https://dlsun.github.io/pods/data/"
df_election = pd.read_csv(data_dir + "election2016.csv")
df_election

*We* will need to merge `df_election` with the `df_counties` `DataFrame` that we defined above. But what do we merge the `DataFrame`s on? It turns out that every county in the United States is assigned a unique ID called a FIPS code. The FIPS code appears in `df_election` as `combined_fips` and in `df_counties` as `GEOID`. Let's take a look at these columns.

In [0]:
df_election.combined_fips

In [0]:
df_counties.GEOID

Notice that `df_counties` treats the FIPS code as a string (so every FIPS code is exactly 5 digits, with a leading zero if necessary). On the other hand, `df_elections` treats the FIPS code as an integer. If we want to join the two, we will have to cast them to the same type. It is probably easier to convert the string to an integer than vice versa.

In [0]:
df_counties["GEOID"] = df_counties["GEOID"].astype(int)

We are almost ready to merge `df_election` and `df_counties`. But before we proceed, remember that _order matters_. We need the order of the data values to match the order of the polygons. The shapefile guarantees that `df_counties` is aligned with `m.counties`, but `df_election` may list the counties in a different order. We need to merge `df_counties` with `df_election` in such a way that preserves the ordering of the counties in `df_counties`. A _left join_ preserves the ordering in the left `DataFrame`, as long as the relationship is not one-to-many. (Each county should appear only once in `df_election`, so this condition holds.)

In [0]:
df_all = df_counties.merge(df_election, 
                           how="left", 
                           left_on="GEOID", right_on="combined_fips")
df_all

One quick sanity check is to make sure that `df` has the same number of rows as `df_counties` (and `m.counties`). This seems to be the case.

Now that the data values are aligned with the counties, we create a choropleth exactly as before:

1. Create a `Polygon` for each county from the vertices in `m.counties`.
2. Create a `PatchCollection` out of the `Polygons`. Set a color map for the collection.
3. Call `PatchCollection.set_array()` on the data values. (Make sure the data values appear in the same order as the polygons.)
4. Draw the map.

There is just one catch. When we left-joined `df_counties` to `df_election`, some of the FIPS codes could not be matched. Therefore, these counties will be missing election data. Unfortunately, `matplotlib`'s color maps do not handle missing values gracefully, so we will have to handle these manually.

Let's first take a look at which states these counties were in.

In [0]:
df_all[df_all.per_dem.isnull()].STATEFP.value_counts()

[A list of FIPS State Codes can be found here](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code). The "states" that had more than one county missing election data are all outlying territories: Puerto Rico (72), Northern Mariana Islands (69), Virgin Islands (78), American Samoa (60), and Guam (66). It turns out that the two remaining states, each with exactly one county that could not be joined, are Alaska (02) and Hawaii (15), which do not show up on our map of the continental United States. So we can safely fill the missing values in `df_all` with an arbitrary value in the middle of the data range without affecting the appearance of the map, since none of these states/territories actually show up on the map.

In [0]:
# Turn each set of vertices into a matplotlib Polygon.
polygons = [mpl.patches.Polygon(vertices) for vertices in m.counties]

# Create a collection of Polygons.
p = mpl.collections.PatchCollection(polygons, cmap="RdBu")

# Set the data that you want to plot.
# (Notice that we fill missing values with 0.5, per the above discussion.)
p.set_array(df_all.per_dem.fillna(0.5))

# Draw the map.
plt.figure(figsize=(12, 8))
m.drawmapboundary()
ax = plt.gca()
ax.add_collection(p)

## Communication Corner

> "Oh, I love those beautiful red areas, that middle of the map.  There’s just a little blue here and a little blue there.  Everything else — everything else is bright red." 
>
> -- Donald Trump

Choropleths can be misleading because they violate the *area principle*---the principle that the area on a graph should represent the magnitude of the data being presented. Since the geographic size of a county is often irrelevant to the data being presented, choropleths can easily be misinterpreted.

For example, President Donald Trump cites the large amount of red area on the 2016 electoral map as an indication of overwhelming support for him. However, the total amount of red area is a statistic that conflates two unrelated quantities: the geographic size of a county and the depth of support for Trump. In fact, most of the red area is in rural parts of the country where few people live. From the choropleth, one would never guess that more people actually voted for his opponent Hillary Clinton! She received strong support from urban areas, but unfortunately for her, cities can barely be seen on a map---despite [being home to a majority of Americans](https://www.census.gov/newsroom/press-releases/2015/cb15-33.html).

It is important to be cautious when designing and interpreting choropleths.

## Exercises

1\. The data set (`https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv`) contains data about the number of confirmed cases of the COVID-19 disease that caused a pandemic in 2020. Download shapefiles of the countries of the world from the [Natural Earth website](https://www.naturalearthdata.com/downloads/), and make a choropleth showing the number of confirmed cases of COVID-19 in each country.